## AI Agents for correcting the questions

In [1]:
# !pip install -U langgraph

Defaulting to user installation because normal site-packages is not writeable
  Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
Using cached httpcore-1.0.7-py3-none-any.whl (78 kB)
Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 27.4 MB/s eta 0:00:00
Using cached PyYAML-6.0.2-cp312-cp312-win_amd64.whl (156 kB)
Using cached tenacity-9.0.0-py3-none-any.whl (28 kB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Us

In [4]:
# !pip install langgraph-checkpoint-sqlite

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage

memory = SqliteSaver.from_conn_string(":memory:")

In [7]:
class AgentState(TypedDict):
    task: str
    critique: str
    revision_number: int # keep track of the number revisions
    max_revisions: int # use for criteria

## Installation for Langchain OpenAI

In [9]:
!pip install langchain_openai

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/883.8 kB ? eta -:--:--
   ---------------------------------------- 883.8/883.8 kB 8.0 MB/s eta 0:00:00


In [10]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [11]:
pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)
